In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import os

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
# from PositionCSVData import GenericCSV_Position
# %matplotlib

ImportError: cannot import name 'GenericCSV_Position' from 'PositionCSVData' (/Users/lucasmoyer/development/two-dolla/backtrader_app/PositionCSVData.py)

In [ ]:
from backtrader.feeds import GenericCSVData

class GenericCSV_Position(GenericCSVData):

    # Add a 'pe' line to the inherited ones from the base class
    lines = ('position',)

    # openinterest in GenericCSVData has index 7 ... add 1
    # add the parameter to the parameters inherited from the base class
    params = (('position', 7),)

# %matplotlib inline
# %matplot%lib widget
# %matplotlib notebook
# %matplotlib qt
# %matplotlib tk
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # self.position = self.datas[0].position
        self.log("INIT CALLED")
        print(self.datas[0])
        
        self.log(self.datas[0].position)

    # def next(self):
    #     # Simply log the closing price of the series from the reference
    #     self.log('Close, %.2f' % self.dataclose[0])
    #     self.log('Prev Close, %.2f' % self.dataclose[1])
    def next(self):
        self.log("NEXT CALLED")
        self.log(self.datas[0].close)
        # self.log(self.position[0])
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # default stake of 1 and executed at markt
                self.buy()
        
if __name__ == '__main__':
    cerebro = bt.Cerebro()


    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, '../../datasets/')
    path = "../datasets/stock_csvs/T.csv"

    data = GenericCSV_Position(
        dataname=path,

        fromdate=datetime.datetime(2023, 6, 1),
        todate=datetime.datetime(2023, 7, 15),

        nullvalue=0.0,

        dtformat=('%Y-%m-%d'),

        datetime=0,
        high=2,
        low=3,
        open=1,
        close=4,
        volume=5,
        openinterest=-1,
        position=7
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-07-14, INIT CALLED
2023-07-14, <backtrader.linebuffer.LineBuffer object at 0x11e9ea050>
2023-06-01, NEXT CALLED
2023-06-01, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-01, Close, 15.81
2023-06-02, NEXT CALLED
2023-06-02, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-02, Close, 15.21
2023-06-05, NEXT CALLED
2023-06-05, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-05, Close, 15.39
2023-06-06, NEXT CALLED
2023-06-06, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-06, Close, 15.67
2023-06-07, NEXT CALLED
2023-06-07, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-07, Close, 16.10
2023-06-08, NEXT CALLED
2023-06-08, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-08, Close, 16.07
2023-06-09, NEXT CALLED
2023-06-09, <backtrader.linebuffer.LineBuffer object at 0x11e9e9d50>
2023-06-09, Close, 15.95
2023-06-09, BUY CREATE, 15.95
2023-06-12, NEX

In [7]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import os

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # To keep track of pending orders
        self.order = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=15)


    # def notify_order(self, order):
    #     if order.status in [order.Submitted, order.Accepted]:
    #         # Buy/Sell order submitted/accepted to/by broker - Nothing to do
    #         return

    #     # Check if an order has been completed
    #     # Attention: broker could reject order if not enough cash
    #     if order.status in [order.Completed]:
    #         if order.isbuy():
    #             self.log('BUY EXECUTED, %.2f' % order.executed.price)
    #         elif order.issell():
    #             self.log('SELL EXECUTED, %.2f' % order.executed.price)

    #         self.bar_executed = len(self)

    #     elif order.status in [order.Canceled, order.Margin, order.Rejected]:
    #         self.log('Order Canceled/Margin/Rejected')

    #     # Write down: no pending order
    #     self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

            # # Not yet ... we MIGHT BUY if ...
            # if self.dataclose[0] < self.dataclose[-1]:
            #         # current close less than previous close

            #         if self.dataclose[-1] < self.dataclose[-2]:
            #             # previous close less than the previous close

            #             # BUY, BUY, BUY!!! (with default parameters)
            #             self.log('BUY CREATE, %.2f' % self.dataclose[0])

            #             # Keep track of the created order to avoid a 2nd order
            #             self.order = self.buy()

        # else:

        #     # Already in the market ... we might sell
        #     if len(self) >= (self.bar_executed + 5):
        #         # SELL, SELL, SELL!!! (with all possible default parameters)
        #         self.log('SELL CREATE, %.2f' % self.dataclose[0])

        #         # Keep track of the created order to avoid a 2nd order
        #         self.order = self.sell()
        
if __name__ == '__main__':
    cerebro = bt.Cerebro()


    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, '../../datasets/')
    path = "./datasets/stock_csvs/T.csv"

    data = btfeeds.GenericCSVData(
        dataname=path,

        fromdate=datetime.datetime(2023, 4, 26),
        todate=datetime.datetime(2023, 7, 15),

        nullvalue=0.0,

        dtformat=('%Y-%m-%d'),

        datetime=0,
        high=2,
        low=3,
        open=1,
        close=4,
        volume=5,
        openinterest=-1
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    # Set the sizer stake from the params
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()

    # print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # %matplotlib ipympl
    # cerebro.plot()
    cerebro.plot()

In [8]:
cerebro.plot(iplot = False)

In [9]:
!pip install ipympl


In [5]:
%matplotlib widget

In [6]:
!conda install tornado=4.5.3
